In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [10]:
LobNamevar=input("Enter the LOBNAME")
speriod=int(input("Enter the simulation period: "))
samples=int(input("Enter the number of samples: "))

In [28]:


# Define the folder containing the Parquet files
folder_path = r'D:\RISHIN\13_ILC_TASK1\input\PARQUET_FILES'

# List all Parquet files in the folder
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Initialize an empty DataFrame to store the results
final_grouped_df = pd.DataFrame()

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Convert the PyArrow Table to a Pandas DataFrame
    df = table.to_pandas()
    
    # Filter the DataFrame where LobName equals LobNamevar
    filtered_df = df[df['LobName'] == LobNamevar]
    
    # Perform the aggregation: sum the Loss column grouped by EventId and PeriodId
    grouped_df = filtered_df.groupby(['EventId', 'PeriodId'], as_index=False).agg({'Loss': 'sum'})
    
    # Rename the aggregated column to Sum_Loss
    grouped_df.rename(columns={'Loss': 'Sum_Loss'}, inplace=True)
    
    # Append the grouped DataFrame to the final_grouped_df
    final_grouped_df = pd.concat([final_grouped_df, grouped_df], ignore_index=True)

# Perform final grouping and sorting
final_grouped_df = final_grouped_df.groupby(['EventId', 'PeriodId'], as_index=False).agg({'Sum_Loss': 'sum'})
final_grouped_df = final_grouped_df.sort_values(by=[ 'Sum_Loss'], ascending=[ False])

# The DataFrame is now ready for the next instructions
dataframe_1 = final_grouped_df

In [29]:
# Initialize dataframe_2 by selecting PeriodId and max(Sum_Loss) grouped by PeriodId
dataframe_2 = dataframe_1.groupby('PeriodId', as_index=False).agg({'Sum_Loss': 'max'})

# Rename the aggregated column to Max_Loss
dataframe_2.rename(columns={'Sum_Loss': 'Max_Loss'}, inplace=True)

# Sort dataframe_2 by Max_Loss in descending order
dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)



In [30]:
# Initialize dataframe_2 by selecting PeriodId and Sum(Sum_Loss) grouped by PeriodId
dataframe_3 = dataframe_1.groupby('PeriodId', as_index=False).agg({'Sum_Loss': 'sum'})

# Rename the aggregated column to Sum_Loss
dataframe_3.rename(columns={'Sum_Loss': 'S_Sum_Loss'}, inplace=True)

# Sort dataframe_3 by S_sum_Loss in descending order
dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)



In [95]:
# Calculate the rate column and round to 6 decimal places
dataframe_2['rate'] = round(1 / (speriod * samples), 6)

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_2['cumrate'] = dataframe_2['rate'].cumsum().round(6)

# Calculate the RPs column and round to 6 decimal places
dataframe_2['RPs'] = (1 / dataframe_2['cumrate']).round(7)

# Calculate the TCE_OEP_1 column and round to 6 decimal places
dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * 
                          (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5).round(6)

# Calculate the TCE_OEP_2 column and round to 6 decimal places
dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].cumsum() * dataframe_2['RPs']).round(6)

# Calculate the TCE_OEP_Final column and round to 6 decimal places
dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss']).round(6)

In [90]:
# Calculate the rate column and round to 6 decimal places
dataframe_3['rate'] = round(1 / (speriod * samples), 6)

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_3['cumrate'] = dataframe_3['rate'].cumsum().round(6)

# Calculate the RPs column and round to 6 decimal places
dataframe_3['RPs'] = (1 / dataframe_3['cumrate']).round(7)
# Calculate the TCE_AEP_1 column and round to 6 decimal places
dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * 
                          (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5).round(2)

# Calculate the TCE_AEP_2 column and round to 6 decimal places
dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].cumsum() * dataframe_3['RPs']).round(2)

# Calculate the TCE_AEP_Final column and round to 6 decimal places
dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss']).round(2)

In [91]:
dataframe_3.head(20)

,PeriodId,S_Sum_Loss,rate,cumrate,RPs,TCE_AEP_1,TCE_AEP_2,TCE_AEP_Final
147788,149148,1.660418e+09,0.000004,0.000004,250000.000000,848.36,2.120900e+08,1.872508e+09
197324,199148,1.519025e+09,0.000004,0.000008,125000.000000,1590.60,3.048700e+08,1.823895e+09
205656,207557,1.359965e+09,0.000004,0.000012,83333.333333,110.57,2.124608e+08,1.572426e+09
4264,4312,1.352067e+09,0.000004,0.000016,62500.000000,1255.64,2.378231e+08,1.589890e+09
152903,154312,1.282309e+09,0.000004,0.000020,50000.000000,1378.47,2.591820e+08,1.541491e+09
103347,104312,1.219651e+09,0.000004,0.000024,41666.666667,735.16,2.466167e+08,1.466268e+09
191303,193067,1.191376e+09,0.000004,0.000028,35714.285714,922.11,2.443182e+08,1.435694e+09
236882,239080,1.160639e+09,0.000004,0.000032,31250.000000,139.70,2.181441e+08,1.378783e+09
48693,49148,1.156530e+09,0.000004,0.000036,27777.777778,477.20,2.071614e+08,1.363692e+09
98237,99148,1.143972e+09,0.000004,0.000040,25000.000000,325.17,1.945745e+08,1.338547e+09


In [92]:
dataframe_3[dataframe_3['PeriodId'] == 76654]

,PeriodId,S_Sum_Loss,rate,cumrate,RPs,TCE_AEP_1,TCE_AEP_2,TCE_AEP_Final
75941,76654,2.866675e+08,0.000004,0.00252,396.825397,158.08,1.497529e+08,4.364205e+08


In [94]:
dataframe_3[dataframe_3['PeriodId'] == 63817]

,PeriodId,S_Sum_Loss,rate,cumrate,RPs,TCE_AEP_1,TCE_AEP_2,TCE_AEP_Final
63238,63817,4.042571e+08,0.000004,0.001,1000.0,548.67,185787700.0,5.900448e+08


In [ ]:
similary i want an another script..so in first case is 1) select PeriodId,EventId,sum(Loss)Sum_Loss where LobName == LobNamevar grouped by PeriodId,EventId ordered by sum_loss DESC
2) select PeriodId,max(Loss)Max_Loss grouped by PeriodId  ordered by Max_Loss DESC from the result of first case 

I will give the third case later